In [185]:
import numpy as np
import torch
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import torch.onnx
from sklearn.model_selection import train_test_split
from random import randint

In [2]:
%matplotlib inline
input_img_sz=11
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="../../../code/bit_2022/data",
    train=True,
    download=True,
    transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize(input_img_sz), torchvision.transforms.ToTensor()]
    ),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="../../../code/bit_2022/data",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize(input_img_sz), torchvision.transforms.ToTensor()]
    ),
)

In [3]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.liner0 = nn.Linear(input_img_sz*input_img_sz, 32)
        self.relu = nn.ReLU(inplace=True)
        self.liner1 =nn.Linear(32, 10)


    def forward(self, x):
        x = self.flatten(x)
        x = self.liner0(x)
        x = self.relu(x)
        x = self.liner1(x)
        return x

model = NeuralNetwork().to(device)
print(model)
loss_fn = nn.CrossEntropyLoss()

# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (liner0): Linear(in_features=121, out_features=32, bias=True)
  (relu): ReLU(inplace=True)
  (liner1): Linear(in_features=32, out_features=10, bias=True)
)
Shape of X [N, C, H, W]: torch.Size([64, 1, 11, 11])
Shape of y: torch.Size([64]) torch.int64


In [4]:
file_name ="MNIST_"+str(input_img_sz)+"x"+str(input_img_sz)+"_32_10.pth"
print(file_name)

MNIST_11x11_32_10.pth


In [5]:
import os
file_name ="MNIST"+str(input_img_sz)+"x"+str(input_img_sz)+"_32_10.pth"
if not (os.path.isfile(file_name)):
    epochs = 10
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
        torch.save(model, file_name)
    print("Done!")
else:
    #model.load_state_dict(torch.load("model.pth"))
    model = torch.load( file_name,map_location=torch.device('cpu'))
    print("Model loaded")

Model loaded


In [6]:
model = torch.load( file_name,map_location=torch.device('cpu'))
for ele in model.state_dict():#turn to numpy
    model.state_dict()[ele]=model.state_dict()[ele].cpu().numpy()
#turn float to int
for layers in model.state_dict():
    try:
        model.state_dict()[layers].shape[1]## if parameter is 2D
        row,col=model.state_dict()[layers].shape
        for row in range(model.state_dict()[layers].shape[0]):
            for col in range(model.state_dict()[layers].shape[1]):
                num=int((model.state_dict()[layers][row][col]+1)/2*255-127)
                model.state_dict()[layers][row][col]= num
    except:
        #model[layers].shape #1D
        # print(model[layers].shape[0])
        for i in range(model.state_dict()[layers].shape[0]):
            num=int((model.state_dict()[layers][i]+1)/2*255-127)
            model.state_dict()[layers][i]=num
test(test_dataloader, model, loss_fn)
model.state_dict()

Test Error: 
 Accuracy: 94.6%, Avg loss: 1621.760395 



OrderedDict([('liner0.weight',
              tensor([[ 19.,  33., -23.,  ...,   0.,   8., -15.],
                      [-21.,  27.,  73.,  ..., -18.,  31.,  57.],
                      [ 18.,  13.,  60.,  ...,  55., -14.,  14.],
                      ...,
                      [-25., -56., -60.,  ..., -24., -12., -24.],
                      [ -9., -35., -55.,  ...,  62., 113.,  25.],
                      [-51., -50., -80.,  ...,  46., -10., -26.]])),
             ('liner0.bias',
              tensor([-26.,  45., -11.,  34.,  58.,  36.,  29.,  20.,  55.,  33., -25.,  -1.,
                       34.,  13.,  39.,  33.,  -9.,  18.,  11.,   1.,  30.,   8.,  45.,  -5.,
                      -33., -28., -15.,   6.,  -9.,  50.,  77., -19.])),
             ('liner1.weight',
              tensor([[  62.,    3.,   62.,   54.,   28., -140., -104.,  -81.,  -66.,  -24.,
                        -96.,   77.,  -66., -161., -130.,   86.,  -22.,  -56.,  -11.,   -1.,
                         62.,   80.,

In [93]:
model = torch.load( file_name,map_location=torch.device('cpu'))
A=59
for ele in model.state_dict():#turn to numpy
    model.state_dict()[ele]=model.state_dict()[ele].cpu().numpy()
#turn float to int
for layers in model.state_dict():
    try:
        model.state_dict()[layers].shape[1]## if parameter is 2D
        row,col=model.state_dict()[layers].shape
        for row in range(model.state_dict()[layers].shape[0]):
            for col in range(model.state_dict()[layers].shape[1]):
                num=int((model.state_dict()[layers][row][col]+1)/2*255-127)
                model.state_dict()[layers][row][col]= round(num/A)*A
    except:
        #model[layers].shape #1D
        # print(model[layers].shape[0])
        for i in range(model.state_dict()[layers].shape[0]):
            num=int((model.state_dict()[layers][i]+1)/2*255-127)
            model.state_dict()[layers][i]=round(num/A)*A
test(test_dataloader, model, loss_fn)
model.state_dict()

Test Error: 
 Accuracy: 89.6%, Avg loss: 3117.378514 



OrderedDict([('liner0.weight',
              tensor([[  0.,  59.,   0.,  ...,   0.,   0.,   0.],
                      [  0.,   0.,  59.,  ...,   0.,  59.,  59.],
                      [  0.,   0.,  59.,  ...,  59.,   0.,   0.],
                      ...,
                      [  0., -59., -59.,  ...,   0.,   0.,   0.],
                      [  0., -59., -59.,  ...,  59., 118.,   0.],
                      [-59., -59., -59.,  ...,  59.,   0.,   0.]])),
             ('liner0.bias',
              tensor([  0.,  59.,   0.,  59.,  59.,  59.,   0.,   0.,  59.,  59.,   0.,   0.,
                       59.,   0.,  59.,  59.,   0.,   0.,   0.,   0.,  59.,   0.,  59.,   0.,
                      -59.,   0.,   0.,   0.,   0.,  59.,  59.,   0.])),
             ('liner1.weight',
              tensor([[  59.,    0.,   59.,   59.,    0., -118., -118.,  -59.,  -59.,    0.,
                       -118.,   59.,  -59., -177., -118.,   59.,    0.,  -59.,    0.,    0.,
                         59.,   59.,

In [8]:
#turn img from float to int
def int_conver(np_a):
    for idx,ele in enumerate(np_a):
        np_a[idx]=int((ele+1)/2*255-127)

    return np_a

In [9]:
#Bit design 
#121 x 32 x 10 
#First layer 17 bit
#Second layer 24 bit

In [10]:
a=np.binary_repr(-5, width=7)
a

'1111011'

In [106]:
hit=0
lrsb0_len=32
lrsb2_len=10
max=0
min=0
lrsw0=model.state_dict()["liner0.weight"].reshape(-1)
lrsb0=model.state_dict()["liner0.bias"].reshape(-1)
lrsw2=model.state_dict()["liner1.weight"].reshape(-1)
lrsb2=model.state_dict()["liner1.bias"].reshape(-1)
for r in range(1):    
    img,label=test_data[i]
    img=img.reshape(-1)
    img=int_conver(img)
    # img = np.zeros(input_img_sz*input_img_sz)
    # img=np.ones(input_img_sz*input_img_sz)
    # img=one(img)
    out1=np.zeros(lrsb0_len)
    out2=np.zeros(lrsb2_len)
    
    for i in range(lrsb0_len):
        for j in range(input_img_sz*input_img_sz):
            out1[i]+=lrsw0[i*input_img_sz*input_img_sz+j]*img[j]
        out1[i]+=lrsb0[i]
    # print(out1)
    for i in range(lrsb0_len):
        if(out1[i]>max):
            max=out1[i]
        if(out1[i]<min):
            min=out1[i]
        if(out1[i]<0):
            out1[i]=0

    for i in range(lrsb2_len):
        for j in range(lrsb0_len):
            out2[i]+=lrsw2[i*lrsb0_len+j]*out1[j]
        out2[i]+=lrsb2[i]

    for i in range(10):
        if(out2[i]>max):
            max=out2[i]
        if(out2[i]<min):
            min=out2[i]
    if(out2.argmax()==label):
        hit+=1

    print("now:",r+1,"hit:",hit)
print(hit)
# print(max, min)
print(len(bin(int(max)))-2,"bit")

now: 1 hit: 1
1
24 bit


In [107]:
out2

array([ -9771167., -32453300.,   -549998.,  13854380., -26145792.,
        -6363209., -34273928.,  16454746.,  -2461126.,   2607269.])

In [98]:
str(out2[0])

'1.101111001110101e+26'

In [88]:
from A59_AN_decoder import A59_AN_decoder
A59_AN_decoder(1,1)

1

In [166]:
bit=27
def twos_com_2_decimal(tows_com):
    if(tows_com[0]!='1'):
        return(int(tows_com,2))
    else:
        return(int(tows_com,2)-(1<<bit))
def binary_repr(decimal):
    return(np.binary_repr(int(decimal), width=bit))
    
# for ele in out2:
#     binary_num = binary_repr(ele)
#     # print(binary_num)
#     deci_num  =twos_com_2_decimal (binary_num)
#     # print(deci_num)
#     if(int(ele)!=deci_num):
#         print(ele ,deci_num)
#         print("Fail!")
def non_repetive_radndom_number(random_range,size):
    a = np.arange(random_range)
    np.random.shuffle(a)
    return (a[:size])


In [197]:
# Second Layer 27 bit
hit=0
lrsb0_len=32
lrsb2_len=10
max=0
min=0
lrsw0=model.state_dict()["liner0.weight"].reshape(-1)
lrsb0=model.state_dict()["liner0.bias"].reshape(-1)
lrsw2=model.state_dict()["liner1.weight"].reshape(-1)
lrsb2=model.state_dict()["liner1.bias"].reshape(-1)
for r in range(100):    
    img,label=test_data[r]
    img=img.reshape(-1)
    img=int_conver(img)
    # img = np.zeros(input_img_sz*input_img_sz)
    # img=np.ones(input_img_sz*input_img_sz)
    # img=one(img)
    out1=np.zeros(lrsb0_len)
    out2=np.zeros(lrsb2_len)
    
    for i in range(lrsb0_len):
        for j in range(input_img_sz*input_img_sz):
            out1[i]+=lrsw0[i*input_img_sz*input_img_sz+j]*img[j]
        out1[i]+=lrsb0[i]
    # print(out1)
    for i in range(lrsb0_len):
        if(out1[i]>max):
            max=out1[i]
        if(out1[i]<min):
            min=out1[i]
        if(out1[i]<0):
            out1[i]=0

    for i in range(lrsb2_len):
        for j in range(lrsb0_len):
            out2[i]+=lrsw2[i*lrsb0_len+j]*out1[j]
        out2[i]+=lrsb2[i]

    # for i in range(10):
    #     if(out2[i]>max):
    #         max=out2[i]
    #     if(out2[i]<min):
    #         min=out2[i]


#turn out2 into binary
    out2_str=[]
    out2_decimal=np.zeros(len(out2))
    for i in range(len(out2)):
        out2_str.append(binary_repr(out2[i]))
### Insert Error ###
    out2_str_all=""
    for ele in out2_str:
        out2_str_all+=ele
    #turn to list, so we can assign to str
    out2_str_all_list=list(out2_str_all)
    #Get ten random number 
    rand_number_array=non_repetive_radndom_number(len(out2_str_all_list),1)
    #bit flip, it will flip ,depends on Bit Error Rate, for now it is 100%
    for error_idx in rand_number_array:
        if(out2_str_all_list[error_idx]=='1'):
            out2_str_all_list[error_idx]='0'
        else:
            out2_str_all_list[error_idx]='1'
    #turn list to str
    out2_back =[]
    for word in range(10):
        s=""
        for word_len in range(27):
            s+=out2_str_all_list[word_len+word*27]
        out2_back.append(s)
    ##################
#turn out2 into decimal 
    for i  in range(len(out2)):
        out2_decimal[i]=(twos_com_2_decimal(out2_back[i]))

    if(out2_decimal.argmax()==label):
        hit+=1
##################
# #turn out2 into decimal 
#     for i  in range(len(out2)):
#         out2_decimal[i]=(twos_com_2_decimal(out2_str[i]))

#Decoder and Correct Phase
    # r = infected_codeword % A
    # q = infected_codeword // A
    # msg = A59_AN_decoder(r,q)

    # if(out2.argmax()==label):
    #     hit+=1

    # print("now:",r+1,"hit:",hit)
print(hit)
# print(max, min)

90


In [200]:
# Second Layer 27 bit
BER= [100,50,25,10,5,1]
point  = [1,10,27,54,81,108,135,162,189,216,243,270]


lrsb0_len=32
lrsb2_len=10
max=0
min=0
test_data_size=1000
lrsw0=model.state_dict()["liner0.weight"].reshape(-1)
lrsb0=model.state_dict()["liner0.bias"].reshape(-1)
lrsw2=model.state_dict()["liner1.weight"].reshape(-1)
lrsb2=model.state_dict()["liner1.bias"].reshape(-1)
for bit_error_rate in BER:
    for error_point_number in point:
        hit=0
        for r in range(test_data_size):    
            img,label=test_data[r]
            img=img.reshape(-1)
            img=int_conver(img)
            # img = np.zeros(input_img_sz*input_img_sz)
            # img=np.ones(input_img_sz*input_img_sz)
            # img=one(img)
            out1=np.zeros(lrsb0_len)
            out2=np.zeros(lrsb2_len)
            
            for i in range(lrsb0_len):
                for j in range(input_img_sz*input_img_sz):
                    out1[i]+=lrsw0[i*input_img_sz*input_img_sz+j]*img[j]
                out1[i]+=lrsb0[i]
            # print(out1)
            for i in range(lrsb0_len):
                if(out1[i]<0):
                    out1[i]=0

            for i in range(lrsb2_len):
                for j in range(lrsb0_len):
                    out2[i]+=lrsw2[i*lrsb0_len+j]*out1[j]
                out2[i]+=lrsb2[i]
        #turn out2 into binary
            out2_str=[]
            out2_decimal=np.zeros(len(out2))
            for i in range(len(out2)):
                out2_str.append(binary_repr(out2[i]))

        ### Insert Error ###
            out2_str_all=""
            for ele in out2_str:
                out2_str_all+=ele
            #turn to list, so we can assign to str
            out2_str_all_list=list(out2_str_all)
            #Get ten random number 
            rand_number_array=non_repetive_radndom_number(len(out2_str_all_list),error_point_number)
            #bit flip, it will flip ,depends on Bit Error Rate

            for error_idx in rand_number_array:
                #bit flip depends on bit error rate
                if(bit_error_rate>randint(1,100)):
                    if(out2_str_all_list[error_idx]=='1'):
                        out2_str_all_list[error_idx]='0'
                    else:
                        out2_str_all_list[error_idx]='1'
            #turn list to str
            out2_back =[]
            for word in range(10):
                s=""
                for word_len in range(27):
                    s+=out2_str_all_list[word_len+word*27]
                out2_back.append(s)
            ##################
        #turn out2 into decimal 
            for i  in range(len(out2)):
                out2_decimal[i]=(twos_com_2_decimal(out2_back[i]))

            if(out2_decimal.argmax()==label):
                hit+=1


        #Decoder and Correct Phase
            # r = infected_codeword % A
            # q = infected_codeword // A
            # msg = A59_AN_decoder(r,q)

            # print("now:",r+1,"hit:",hit)
        print("BER:",bit_error_rate,"error point ",error_point_number,hit/test_data_size)


BER: 100 error point  1 0.824
BER: 100 error point  10 0.538
BER: 100 error point  27 0.277
BER: 100 error point  54 0.135
BER: 100 error point  81 0.092
BER: 100 error point  108 0.081
BER: 100 error point  135 0.093
BER: 100 error point  162 0.112
BER: 100 error point  189 0.119
BER: 100 error point  216 0.124
BER: 100 error point  243 0.065
BER: 100 error point  270 0.004
BER: 50 error point  1 0.829
BER: 50 error point  10 0.676
BER: 50 error point  27 0.458
BER: 50 error point  54 0.299
BER: 50 error point  81 0.196
BER: 50 error point  108 0.145
BER: 50 error point  135 0.11
BER: 50 error point  162 0.102
BER: 50 error point  189 0.094
BER: 50 error point  216 0.096
BER: 50 error point  243 0.121
BER: 50 error point  270 0.087
BER: 25 error point  1 0.845
BER: 25 error point  10 0.761
BER: 25 error point  27 0.634
BER: 25 error point  54 0.486
BER: 25 error point  81 0.366
BER: 25 error point  108 0.315
BER: 25 error point  135 0.248
BER: 25 error point  162 0.224
BER: 25 error p

In [204]:
# Second Layer 27 bit
BER= [100,50,25,10,5,1]
point  = [1,10,27,54,81,108,135,162,189,216,243,270]

save_string=""
lrsb0_len=32
lrsb2_len=10
max=0
min=0
test_data_size=1000
lrsw0=model.state_dict()["liner0.weight"].reshape(-1)
lrsb0=model.state_dict()["liner0.bias"].reshape(-1)
lrsw2=model.state_dict()["liner1.weight"].reshape(-1)
lrsb2=model.state_dict()["liner1.bias"].reshape(-1)
for bit_error_rate in BER:
    for error_point_number in point:
        hit=0
        for r in range(test_data_size):    
            img,label=test_data[r]
            img=img.reshape(-1)
            img=int_conver(img)
            # img = np.zeros(input_img_sz*input_img_sz)
            # img=np.ones(input_img_sz*input_img_sz)
            # img=one(img)
            out1=np.zeros(lrsb0_len)
            out2=np.zeros(lrsb2_len)
            
            for i in range(lrsb0_len):
                for j in range(input_img_sz*input_img_sz):
                    out1[i]+=lrsw0[i*input_img_sz*input_img_sz+j]*img[j]
                out1[i]+=lrsb0[i]
            # print(out1)
            for i in range(lrsb0_len):
                if(out1[i]<0):
                    out1[i]=0

            for i in range(lrsb2_len):
                for j in range(lrsb0_len):
                    out2[i]+=lrsw2[i*lrsb0_len+j]*out1[j]
                out2[i]+=lrsb2[i]
        #turn out2 into binary
            out2_str=[]
            out2_decimal=np.zeros(len(out2))
            for i in range(len(out2)):
                out2_str.append(binary_repr(out2[i]))

        ### Insert Error ###
            out2_str_all=""
            for ele in out2_str:
                out2_str_all+=ele
            #turn to list, so we can assign to str
            out2_str_all_list=list(out2_str_all)
            #Get ten random number 
            rand_number_array=non_repetive_radndom_number(len(out2_str_all_list),error_point_number)
            #bit flip, it will flip ,depends on Bit Error Rate

            for error_idx in rand_number_array:
                #bit flip depends on bit error rate
                if(bit_error_rate>randint(1,100)):
                    if(out2_str_all_list[error_idx]=='1'):
                        out2_str_all_list[error_idx]='0'
                    else:
                        out2_str_all_list[error_idx]='1'
            #turn list to str
            out2_back =[]
            for word in range(10):
                s=""
                for word_len in range(27):
                    s+=out2_str_all_list[word_len+word*27]
                out2_back.append(s)
            ##################
        #turn out2 into decimal 
            for i  in range(len(out2)):
                out2_decimal[i]=(twos_com_2_decimal(out2_back[i]))

            for idx,infected_codeword in enumerate(out2_decimal):
                r = infected_codeword % A
                q = int(infected_codeword // A)
                if(r!=0):
                    msg = A59_AN_decoder(int(r),int(q))
                    out2_decimal[idx]=msg
                else:
                    out2_decimal[idx]=q
            if(out2_decimal.argmax()==label):
                hit+=1


        #Decoder and Correct Phase
            # r = infected_codeword % A
            # q = infected_codeword // A
            # msg = A59_AN_decoder(r,q)

            # print("now:",r+1,"hit:",hit)
        print("BER:",bit_error_rate,"error point ",error_point_number,hit/test_data_size)
        save_string+="BER:      "+str(bit_error_rate)+"error:       "+str(error_point_number)+"accuracy:        "+str(hit/test_data_size)+"\n"
with open("result_of_A59_BER_accuracy.txt","w") as f:
    f.write(save_string)

BER: 100 error point  1 0.845
BER: 100 error point  10 0.525
BER: 100 error point  27 0.192
BER: 100 error point  54 0.116
BER: 100 error point  81 0.113
BER: 100 error point  108 0.103
BER: 100 error point  135 0.109
BER: 100 error point  162 0.094
BER: 100 error point  189 0.105
BER: 100 error point  216 0.095
BER: 100 error point  243 0.075
BER: 100 error point  270 0.023
BER: 50 error point  1 0.85
BER: 50 error point  10 0.719
BER: 50 error point  27 0.434
BER: 50 error point  54 0.2
BER: 50 error point  81 0.13
BER: 50 error point  108 0.106
BER: 50 error point  135 0.101
BER: 50 error point  162 0.118
BER: 50 error point  189 0.116
BER: 50 error point  216 0.089
BER: 50 error point  243 0.096
BER: 50 error point  270 0.097
BER: 25 error point  1 0.851
BER: 25 error point  10 0.799
BER: 25 error point  27 0.672
BER: 25 error point  54 0.454
BER: 25 error point  81 0.28
BER: 25 error point  108 0.226
BER: 25 error point  135 0.17
BER: 25 error point  162 0.161
BER: 25 error point 

In [152]:
temp=""
for ele in out2_str:
    temp+=ele
temp

'110111100111010100010100100101101110111100001101110100110111011001101110000100100111111011001000111101000010111111000101001001011100110110111010001011011111000000110100101010110000011001110000011001111110010100111111111111101000010011101111110001000110001111101110011010'

In [162]:
a = np.arange(270)
np.random.shuffle(a)
print (a[:10])

[186 133  47 109 153 253  57 181  70  46]


In [169]:
tmep_list = list(temp)
~tmep_list[0]

TypeError: bad operand type for unary ~: 'str'

In [138]:
print(out2_str[0])
print(out2_decimal[0])
codeword=list(out2_str[0])
codeword[-1] ="1"
s=""
for ele in codeword:
    s+=ele
s

110111100111010100010100100
-17586012.0


'110111100111010100010100101'

In [143]:
infected_codeword=twos_com_2_decimal(s)
print(infected_codeword)
print("msg=",out2_decimal[0]/A)

-17586011
msg= -298068.0


In [141]:
r = infected_codeword % A
q = infected_codeword // A
A59_AN_decoder(r,q)

-298068

In [11]:
layer_out0=""
for ele in reversed(out1):
    layer_out0+="{:020X}".format(int(str(int(ele)),10))
layer_out0

'000000000000000000000000000000000000004D000000000000000000320000000000000000000000000000000000000006000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002D000000000000000000080000000000000000001E000000000000000000010000000000000000000B000000000000000000120000000000000000000000000000000000000021000000000000000000270000000000000000000D0000000000000000002200000000000000000000000000000000000000000000000000000000002100000000000000000037000000000000000000140000000000000000001D000000000000000000240000000000000000003A00000000000000000022000000000000000000000000000000000000002D00000000000000000000'

In [12]:
s="{:05b}".format(int(-5))[1:]
print(s)
p=""
bit_num=10
for i in range(len(s)):
    if(s[i]=="1"):
        p+="0"
    else:
        p+="1"
print(p)
print(int(p,2)+1)
p=int(p,2)+1
print(bin(p)[2:])

0101
1010
11
1011


In [13]:
def two_s_com(num):
    p=""
    if(num<0):
        bin_num="{:05b}".format(int(num))[1:]
        for i in range(len(bin_num)):
            if(bin_num[i]=="1"):
                p+="0"
            else:
                p+="1"
        p=int(p,2)+1
    return (bin(p)[2:])

In [14]:
"{:05b}".format(int(-5))

'-0101'

In [15]:
two_s_com(-5)

'1011'

In [20]:
import TCB
def D2reg(num,reg_name):
    if(num==0):
        return None
    else:
        tcb_str=TCB.Bin2TCB(int(num))
        temp=0
        if(num>0):
            out_s="+(0"
        else:
            out_s="-(0"
        for s in reversed(tcb_str):
            if(s=="+"):
                out_s+="+("+reg_name+"<<"+str(temp)+")"
            if(s=="-"):
                out_s+="-("+reg_name+"<<"+str(temp)+")"
            temp+=1         
            
        return out_s+")"
D2reg(-29,"reg")        


'-(0+(reg<<0)-(reg<<2)+(reg<<5))'

In [21]:
img_size=input_img_sz*input_img_sz
wire_cnt=32
in_buffer_weight_name="in_buffer_weight"
weight_to_bias_name="weight_bias"
bias_relu_name="bias_relu"
in_buffer_weight_parameter=model.state_dict()["liner0.weight"]
weight_bias_parameter=model.state_dict()["liner0.bias"]
data_bit_num=20
output_num=32
layer0_verilog_file_name="layer0_tcb_"+str(img_size)+"x32x10"
f=open("./verilog_test/"+layer0_verilog_file_name+".v","w")
f.write("module "+layer0_verilog_file_name+"\n")
f.write("(\n")

f.write("\
    input clk,\n\
    input rst,\n\
    input ["+str(img_size)+"*8-1:0] img,\n\
    input valid,\n\
    output  reg ready,\n\
    output ["+str(data_bit_num)+"*"+str(output_num)+"-1:0] layer_out\n"\
)
f.write(");\n")
f.write("parameter DATA_WIDTH = "+str(data_bit_num)+";\n")
f.write("parameter IMG_SZ   =   "+str(img_size)+";\n")

f.write("reg    signed [8-1:0]  in_buffer[0:IMG_SZ-1];\n")
f.write("integer i;\n")
f.write("always@(posedge clk )\n\
    begin\n\
        if(rst)\n\
            begin\n\
                for(i=0;i<IMG_SZ;i=i+1)\n\
                    begin\n\
                        in_buffer[i]<=0;\n\
                    end\n\
            end\n\
        else\n\
        begin\n")

for i in range(img_size):
    f.write("       in_buffer["+str(i)+"]<=img["+str(7+i*8)+":"+str(0+i*8)+"];\n")           
f.write("        end\n   end\n")
for naming_idx in range(wire_cnt): 
    f.write("wire signed  [DATA_WIDTH-1:0]   ")   
    f.write(in_buffer_weight_name+str(naming_idx)+";\n")
    f.write("assign "+in_buffer_weight_name+str(naming_idx)+"=0")
    #insert tcb
    for in_buf_idx in range(img_size):
        name="in_buffer[" +str(in_buf_idx)+"]"
        num=int(in_buffer_weight_parameter[naming_idx][in_buf_idx])
        try:
            f.write(D2reg(num,name))
        except:
            pass
    f.write(";\n")

#weight0 to bias0
previous_layer_name=in_buffer_weight_name
#naming wire
print("wire_cnt",wire_cnt)
for naming_idx in range(wire_cnt): 
    f.write("wire signed [DATA_WIDTH-1:0]   ")   
    f.write(weight_to_bias_name+str(naming_idx)+";\n")
for naming_idx in range(wire_cnt):
    f.write("assign "+weight_to_bias_name+str(naming_idx)+"=")
    f.write(previous_layer_name+str(naming_idx))
    f.write("+(")
    f.write(str(int(weight_bias_parameter[naming_idx])))

    f.write(");\n")
#bias0 to relu0
previous_layer_name=weight_to_bias_name
#naming wire
print("wire_cnt",wire_cnt)
for naming_idx in range(wire_cnt): 
    f.write("wire signed [DATA_WIDTH-1:0]   ")   
    f.write(bias_relu_name+str(naming_idx)+";\n")
for naming_idx in range(wire_cnt):
    f.write("assign "+bias_relu_name+str(naming_idx)+"=(")
    f.write(previous_layer_name+str(naming_idx))
    f.write("[DATA_WIDTH-1]==1'b1)   ?   ")
    f.write("{DATA_WIDTH{1'b0}}:")
    f.write(previous_layer_name+str(naming_idx))
    f.write(";\n")


f.write("assign layer_out={")
for naming_idx in range(wire_cnt-1,-1,-1):
    if(naming_idx==0):
        f.write(bias_relu_name+str(naming_idx))
    else:
        f.write(bias_relu_name+str(naming_idx)+",")
f.write("};\n")
f.write("always@(posedge clk)\n\
    begin\n\
        if(rst)\n\
            begin\n\
                ready<=1'b0;\n\
            end\n\
        else\n\
            begin\n\
                ready<=valid;\n\
            end\n\
    end\n")
f.write("endmodule")
f.close()
f.close()

wire_cnt 32
wire_cnt 32


In [22]:
last_output_sz=32
last_layer_data_sz=20
layer2_bit=30
previous_layer_sz=32
wire_cnt=10
in_buffer_weight_name="in_buffer_weight"
weight_to_bias_name="weight_bias"
bias_relu_name="bias_relu"
in_buffer_weight_parameter=model.state_dict()["liner1.weight"]
weight_bias_parameter=model.state_dict()["liner1.bias"]



layer2_verilog_file_name="layer2_tcb_"+str(img_size)+"x32x10"
f=open("./verilog_test/"+layer2_verilog_file_name+".v","w")
f.write("module "+layer2_verilog_file_name+"\n")
f.write("(\n")

f.write("\
    input clk,\n\
    input rst,\n")
f.write("   input valid,\n")
f.write("   output  reg ready,\n")
f.write("    input ["+str(last_layer_data_sz)+"*"+str(last_output_sz)+"-1:0]  layer_in,\n")
f.write("    output ["+str(layer2_bit)+"*10-1:0]   layer_out\n\
);\n")
f.write("parameter DATA_WIDTH   =   "+str(layer2_bit)+";\n")
f.write("reg [DATA_WIDTH-1:0]    layer_in_buffer    [0:"+str(last_output_sz)+"-1];\n")
f.write("integer i;\n")
f.write("always@(posedge clk )\n\
    begin\n\
        if(rst)\n\
            begin\n")
f.write("                for(i=0;i<"+str(last_output_sz)+";i=i+1)\n\
                    begin\n\
                        layer_in_buffer[i]<=0;\n\
                    end\n\
            end\n\
        else\n\
        begin\n")
for i in range(last_output_sz):
    f.write("       layer_in_buffer["+str(i)+"]<=layer_in["+str(last_layer_data_sz-1+i*last_layer_data_sz)+":"+str(0+i*last_layer_data_sz)+"];\n")           
f.write("        end\n   end\n")


f.write("\n")
previous_layer_name="layer_in_buffer"
#naming wire
for naming_idx in range(wire_cnt): 
    f.write("wire [DATA_WIDTH-1:0]   ")   
    f.write(in_buffer_weight_name+str(naming_idx)+";\n")
    f.write("assign "+in_buffer_weight_name+str(naming_idx)+"=0")
    for previoud_layer_idx in range(last_output_sz):
        name=previous_layer_name+"["+str(previoud_layer_idx)+"]"
        num=int(in_buffer_weight_parameter[naming_idx][previoud_layer_idx])
        try:
            f.write(D2reg(num,name))
        except:
            pass
    f.write(";\n")
# weight4 to bias4 

previous_layer_name=in_buffer_weight_name
#naming wire
for naming_idx in range(wire_cnt): 
    f.write("wire [DATA_WIDTH-1:0]   ")   
    f.write(weight_to_bias_name+str(naming_idx)+";\n")
    f.write("assign "+weight_to_bias_name+str(naming_idx)+"=")
    f.write(previous_layer_name+str(naming_idx))
    f.write("+(")
    f.write(str(int(weight_bias_parameter[naming_idx])))
    f.write(");\n")
f.write("assign layer_out={\n")
for naming_idx in range(wire_cnt-1,-1,-1): 
    if(naming_idx==0):
        f.write("            "+weight_to_bias_name+str(naming_idx))
    else:
        f.write("            "+weight_to_bias_name+str(naming_idx)+",\n")
f.write("};\n")
f.write("always@(posedge clk)\n\
    begin\n\
        if(rst)\n\
            begin\n\
                ready<=1'b0;\n\
            end\n\
        else\n\
            begin\n\
                ready<=valid;\n\
            end\n\
    end\n")
f.write("endmodule\n")
f.close()

In [23]:
data_sz=30
f=open("./verilog_test/comparator_30bit.v","w")
f.write("module comparator\n")
f.write("(\n")
f.write("input ["+str(data_sz)+"*10-1:0] layer_out,\n")
f.write("input rst,\n")
f.write("input clk,\n")
f.write("input valid,\n")
f.write("output  reg ready,\n")
f.write("output reg [7:0] predict\n")
f.write(");\n")
f.write("parameter DATA_WIDTH="+str(data_sz)+";\n")
f.write("reg [DATA_WIDTH-1: 0] result [0:9];\n")
f.write("wire [4+DATA_WIDTH-1:0] com_re01,com_re23,com_re45,com_re67,com_re89;\n")
f.write("reg ready_temp;\n")
f.write("always@(posedge clk)\n\
    begin\n\
        if(rst)\n\
            begin\n\
                ready<=1'b0;\n\
                ready_temp<=1'b0;\n\
            end\n\
        else\n\
            begin\n\
                ready_temp<=valid;\n\
                ready<=ready_temp;\n\
            end\n\
    end\n")

f.write(" \n\
assign com_re01=(result[0][DATA_WIDTH-1]^result[1][DATA_WIDTH-1]) ? \n\
                                                        ((result[0][DATA_WIDTH-1]==1'b0)   ?   {4'd0,result[0]}:{4'd1,result[1]}):\n\
                                                        ((result[0]>result[1]) ? {4'd0,result[0]}:{4'd1,result[1]});\n\
assign com_re23=(result[2][DATA_WIDTH-1]^result[3][DATA_WIDTH-1]) ? \n\
                                                        ((result[2][DATA_WIDTH-1]==1'b0)   ?   {4'd2,result[2]}:{4'd3,result[3]}):\n\
                                                        ((result[2]>result[3]) ? {4'd2,result[2]}:{4'd3,result[3]});\n\
assign com_re45=(result[4][DATA_WIDTH-1]^result[5][DATA_WIDTH-1]) ? \n\
                                                        ((result[4][DATA_WIDTH-1]==1'b0)   ?   {4'd4,result[4]}:{4'd5,result[5]}):\n\
                                                        ((result[4]>result[5]) ? {4'd4,result[4]}:{4'd5,result[5]});\n")  
f.write("\n\
assign com_re67=(result[6][DATA_WIDTH-1]^result[7][DATA_WIDTH-1]) ? \n\
                                                        ((result[6][DATA_WIDTH-1]==1'b0)   ?   {4'd6,result[6]}:{4'd7,result[7]}):\n\
                                                        ((result[6]>result[7]) ? {4'd6,result[6]}:{4'd7,result[7]});\n\
assign com_re89=(result[8][DATA_WIDTH-1]^result[9][DATA_WIDTH-1]) ? \n\
                                                        ((result[8][DATA_WIDTH-1]==1'b0)   ?   {4'd8,result[8]}:{4'd9,result[9]}):\n\
                                                        ((result[8]>result[9]) ? {4'd8,result[8]}:{4'd9,result[9]});\n\
wire [4+DATA_WIDTH-1:0] com_re01_23,com_re45_67,com_re0123_4567,com_re01234567_89;\n\
assign com_re01_23=(com_re01[DATA_WIDTH-1]^com_re23[DATA_WIDTH-1])   ?\n\
                                                        ((com_re01[DATA_WIDTH-1]==1'b0)  ?   com_re01:com_re23):\n\
                                                        ((com_re01[DATA_WIDTH-1:0]>com_re23[DATA_WIDTH-1:0]) ?   com_re01:com_re23);\n\
assign com_re45_67=(com_re45[DATA_WIDTH-1]^com_re67[DATA_WIDTH-1])   ?\n\
                                                        ((com_re45[DATA_WIDTH-1]==1'b0)  ?   com_re45:com_re67):\n\
                                                        ((com_re45[DATA_WIDTH-1:0]>com_re67[DATA_WIDTH-1:0]) ?   com_re45:com_re67);\n\
assign com_re0123_4567=(com_re01_23[DATA_WIDTH-1]^com_re45_67[DATA_WIDTH-1])   ?\n\
                                                        ((com_re01_23[DATA_WIDTH-1]==1'b0)  ?   com_re01_23:com_re45_67):\n\
                                                        ((com_re01_23[DATA_WIDTH-1:0]>com_re45_67[DATA_WIDTH-1:0]) ?   com_re01_23:com_re45_67);\n\
assign com_re01234567_89=(com_re0123_4567[DATA_WIDTH-1]^com_re89[DATA_WIDTH-1])   ?\n\
                                                        ((com_re0123_4567[DATA_WIDTH-1]==1'b0)  ?   com_re0123_4567:com_re89):\n\
                                                        ((com_re0123_4567[DATA_WIDTH-1:0]>com_re89[DATA_WIDTH-1:0]) ?   com_re0123_4567:com_re89);\n")
f.write("integer i;\n")
f.write("always@(posedge clk )\n")
f.write("begin\n\
    if(rst)\n\
            begin\n\
                for(i=0;i<10;i=i+1)\n\
                    begin\n\
                        result[i]<={"+str(data_sz)+"'b0};\n\
                    end\n\
                predict<=0;\n\
            end\n\
    else\n\
    begin \n")
f.write("\
        predict <={4'b0,com_re01234567_89[4+DATA_WIDTH-1:4+DATA_WIDTH-1-3]};\n")
for i in range(10):
    f.write("       result["+str(i)+"]<=layer_out["+str(data_sz-1+data_sz*i)+":"+str(data_sz*i)+"];\n")
f.write("\
    end\n\
end\n")



f.write("endmodule\n")
f.close()

In [24]:
layer0_bit=20
layer0_port=32
layer2_port=10
tb_name="top_tcb_"+str(input_img_sz*input_img_sz)+"32_10"
f=open("./verilog_test/"+tb_name+".v","w")
f.write("module "+tb_name+"(\n")
f.write("input clk,\n\
input rst,\n\
input ["+str(input_img_sz*input_img_sz)+"*8-1:0] img_source,\n\
output [7:0] number,\n")
f.write("input valid_top,\n")
f.write("output  ready_top\n")
f.write(");\n")
f.write("wire layer0_ready,layer2_ready\n;")
f.write("wire   ["+str(layer0_bit)+"*"+str(layer0_port)+"-1:0] layer0_out;\n")
f.write("wire   ["+str(layer2_bit)+"*"+str(layer2_port)+"-1:0] layer2_out;\n")
f.write(layer0_verilog_file_name+" DUT_layer0   (.clk(clk),.rst(rst),.img(img_source),.layer_out(layer0_out),.ready(layer0_ready),.valid(valid_top));\n")
f.write(layer2_verilog_file_name+" DUT_layer2   (.clk(clk),.rst(rst),.layer_in(layer0_out),.layer_out(layer2_out),.ready(layer2_ready),.valid(layer0_ready));\n")
f.write("comparator DUT_comparator (.clk(clk),.rst(rst),.layer_out(layer2_out),.predict(number),.ready(ready_top),.valid(layer2_ready));\n")

f.write("endmodule\n")
f.close()

In [25]:

f=open("./verilog_test/tb.v","w")
f.write("`timescale 1ns/1ps\n")
f.write("module tb;\n")

f.write("reg clk,rst,valid;\n")
f.write("reg ["+str(input_img_sz*input_img_sz*8)+"-1:0] img;\n")
f.write("wire [7:0] number;\n")
f.write(tb_name+" top_DUT(\n\
    .clk(clk),\n\
    .rst(rst),\n\
    .img_source(img),\n\
    .valid_top(valid),\n\
    .ready_top(ready_top),\n\
    .number(number)\n\
);\n")
f.write("always #5 clk=~clk;\n")
f.write("initial \nbegin\n")
f.write("clk=0;rst=1'b1;valid=1'b1;\n")
f.write("img="+str(input_img_sz*input_img_sz)+"'b0;\n")
f.write("#10 rst=1'b0;\n")
for i in range(10):
    f.write("@(negedge clk) #(10/4) img="+str(input_img_sz*input_img_sz*8)+"'b")
    img,idx=test_data[i]
    img=img.reshape(-1)
    img=np.asarray(img)
    img=int_conver(img)
    for ele in reversed(img):
        eight_bit="{:08b}".format(int(ele))
        f.write(eight_bit)
        #print(ele)
    f.write(";\n")



f.write("#100 $finish;\n")


f.write("end\n")




f.write("endmodule\n")
f.close()

In [26]:
f=open("./verilog_test/label.txt","w")
for i in range(10):
    img,idx=test_data[i]
    f.write(str(idx))
    f.write("\n")
f.close()

In [27]:
f=open("./verilog_test/half_img.txt","w")
for i in range(10):
    img,idx=test_data[i]
    img=img.reshape(-1)
    img=np.asarray(img)
    img=int_conver(img)
    half_img0=img[0:128]
    half_img1=img[128:256]
    f.write("$display(\""+str(idx)+"\");\n")
    f.write("@(s_axis_ready ==1'b1);")
    f.write("begin @(posedge clk) img_half_part="+str(int(input_img_sz*input_img_sz*8/2))+"'b")
    for ele in reversed(half_img1):
        eight_bit="{:08b}".format(int(ele))
        f.write(eight_bit)
    f.write(";\n")
    f.write("s_axis_last=1'b0;\n")
    f.write("@(posedge clk) img_half_part="+str(int(input_img_sz*input_img_sz*8/2))+"'b")
    for ele in reversed(half_img0):
        eight_bit="{:08b}".format(int(ele))
        f.write(eight_bit)
    f.write(";\n")
    f.write("s_axis_last=1'b1;end\n")
f.close()

In [28]:
f=open("./verilog_test/half_img.txt","w")
for i in range(10):
    img,idx=test_data[i]
    img=img.reshape(-1)
    img=np.asarray(img)
    img=int_conver(img)
    half_img0=img[0:128]
    half_img1=img[128:256]
    f.write("$display(\""+str(idx)+"\");\n")
    f.write("wait (s_axis_ready ==1'b1);")
    f.write("begin @(posedge clk) img_half_part="+str(int(input_img_sz*input_img_sz*8/2))+"'b")
    for ele in reversed(half_img1):
        eight_bit="{:08b}".format(int(ele))
        f.write(eight_bit)
    f.write(";\n")
    f.write("s_axis_last=1'b0;end\n")
    f.write("wait (s_axis_ready ==1'b1);begin @(posedge clk) img_half_part="+str(int(input_img_sz*input_img_sz*8/2))+"'b")
    for ele in reversed(half_img0):
        eight_bit="{:08b}".format(int(ele))
        f.write(eight_bit)
    f.write(";\n")
    f.write("s_axis_last=1'b1;end\n")
f.close()

In [29]:
f=open("./verilog_test/test1.txt","w") #Correct img order
for i in range(1):
    img,idx=test_data[i]
    img=img.reshape(-1)
    img=np.asarray(img)
    img=int_conver(img)
    f.write("$display(\""+str(idx)+"\");\n")
    f.write("wait (s_axis_ready ==1'b1);@(negedge clk) #(10/4) img_half_part="+str(int(input_img_sz*input_img_sz*8))+"'b")
    for ele in reversed(img):
        eight_bit="{:08b}".format(int(ele))
        f.write(eight_bit)
    f.write(";\n")
f.close()

In [30]:
f=open("./verilog_test/test2.txt","w")
for i in range(1):
    img,idx=test_data[i]
    img=img.reshape(-1)
    img=np.arange(0,256)
    img=np.asarray(img)
    #img=int_conver(img)
    half_img0=img[0:128]
    half_img1=img[128:256]
    f.write("$display(\""+str(idx)+"\");\n")
    f.write("wait (s_axis_ready ==1'b1);@(negedge clk) #(10/4) img_half_part="+str(int(input_img_sz*input_img_sz*8/2))+"'b")
    for ele in reversed(half_img0):
        eight_bit="{:08b}".format(int(ele))
        f.write(eight_bit)
    f.write(";\n")
    f.write("s_axis_last=1'b0;\n")
    f.write("wait (s_axis_ready ==1'b1);@(negedge clk) #(10/4) img_half_part="+str(int(input_img_sz*input_img_sz*8/2))+"'b")
    for ele in reversed(half_img1):
        eight_bit="{:08b}".format(int(ele))
        f.write(eight_bit)
    f.write(";\n")
    f.write("s_axis_last=1'b1;\n")
    
f.close()

In [31]:
f=open("./verilog_test/test1.txt","w")

img=np.arange(0,256)
half_img0=img[0:128]
half_img1=img[128:256]
for ele in reversed(half_img1):
    f.write(str(ele))
for ele in reversed(half_img0):
    f.write(str(ele))
f.write("\n")
for ele in reversed(img):
    f.write(str(ele))
f.close()

In [32]:
f=open("./verilog_test/wd32.txt","w")
# f.write("@(s_axis_ready==1'b1)begin\n")
for i in range(2):
    f.write("@(s_axis_ready==1'b1)begin\n")
    img,idx=test_data[i]
    img=img.reshape(-1)
    img=np.asarray(img)
    img=int_conver(img)
    # img=np.arange(0,256)
    for j in range(63,-1,-1):
        img_4word=img[0+j*4:4+j*4]
        f.write("@(negedge clk)#(10/4)begin  img_half_part=32'b")
        for ele in reversed (img_4word):
            eight_bit="{:08b}".format(int(ele))
            f.write(str(eight_bit))
        f.write(";end\n")
    f.write("end\n")
f.close()
print(len(img))

121


In [33]:
len(img)

121

In [34]:
f=open("./verilog_test/cpp_input_img.mem","w") #right one
# f.write("@(s_axis_ready==1'b1)begin\n")
for i in range(1):
    img,idx=test_data[i]
    print(idx)
    img=img.reshape(-1)
    img=np.asarray(img)
    img=int_conver(img)
    # img=np.arange(0,256)
    # f.write("u32 a[32]={\n")

    for ele in (img):
        # print(ele)
        eight_bit="{:08b}".format(int(ele))
        f.write(str(eight_bit))
        f.write("\n")
    for i in range(7):
        eight_bit="{:08b}".format(int(0))
        f.write(str(eight_bit))
        f.write("\n")  
    
    # f.write("};\n")
f.close()


7


In [35]:
f=open("./verilog_test/cpp_input_img.mem","w") ##
# f.write("@(s_axis_ready==1'b1)begin\n")
for i in range(1):
    img,idx=test_data[i]
    print(idx)
    img=img.reshape(-1)
    img=np.asarray(img)
    img=int_conver(img)

    for ele in (img):
        print(ele)
        eight_bit="{:08b}".format(int(ele))
        f.write(str(eight_bit))
        f.write("\n")
    for i in range(7):
        eight_bit="{:08b}".format(int(0))
        f.write(str(eight_bit))
        f.write("\n")  
    

f.close()


7
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
13.0
24.0
12.0
6.0
6.0
6.0
2.0
0.0
0.0
0.0
0.0
41.0
72.0
77.0
75.0
77.0
81.0
39.0
0.0
0.0
0.0
0.0
3.0
5.0
12.0
17.0
25.0
83.0
36.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
39.0
77.0
8.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
7.0
72.0
30.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
52.0
69.0
4.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
24.0
85.0
21.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
77.0
70.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.0
63.0
28.0
1.0
0.0
0.0
0.0
0.0


In [36]:


f=open("./verilog_test/write.txt","w")
f.write("u32 a[32]={")
for i in range(1):
    t=[]
    img,idx=test_data[1]
    print(idx)
    img=img.reshape(-1)
    img=np.asarray(img)
    img=int_conver(img)
    
    for ele in (img):
        t.append(ele)

    for i in range(7):
        t.append(0)
    temp = np.asarray(t)
    temp=temp.reshape((32,4))
    for i in range(32):
        temp[i]=np.flip(temp[i],0)
    #one img done
    w_input=[]
    for word_4 in temp:
        s=""
        for num in word_4:
            eight_bit="{:08b}".format(int(num))
            s+=eight_bit
        decimal_num=str(int(s,2))
        w_input.append(decimal_num)

    for i,e in enumerate(w_input):
        f.write(str(e))
        if(i==(len(w_input)-1)):
            f.write(",")
            # pass
        else:
            f.write(",")
    f.write("\n")
f.write("};")
f.close()

2


In [37]:
w_input=[]

for word_4 in temp:
    s=""
    for num in word_4:
        eight_bit="{:08b}".format(int(num))
        s+=eight_bit
    decimal_num=str(int(s,2))
    w_input.append(decimal_num)

f=open("./verilog_test/write.txt","w")
f.write("u32 a[320]={")
for i,e in enumerate(w_input):
    f.write(str(e))
    if(i==(len(w_input)-1)):
        pass
    else:
        f.write(",")
f.write("};")
f.close()